In [1]:
import pandas as pd
import random 
import time
from bs4 import BeautifulSoup
import requests
from fake_useragent import FakeUserAgent

In [474]:
# Proxies 

proxies = []
response = requests.get('https://vpnhack.com/premium-proxy-list')
soup = BeautifulSoup(response.text, 'lxml')
for element in soup.find('tbody').find_all('tr'):
    ip = element.find('td').text
    port = element.findAll('td')[1].text
    port = port.replace(', 3128','')
    proxies.append(ip+':'+port)

prox = random.choice(proxies)


In [475]:
prox

'128.199.150.242:8080'

In [480]:
from fake_useragent import FakeUserAgent

ua = FakeUserAgent().random

def get_soup(link):
    r = requests.get(link, headers = {'headers':ua}, proxies = {'http':prox}, timeout =5)
    soup = BeautifulSoup(r.text,'lxml')

    return soup

In [728]:
# Les notes 

def get_data(soup):
    
    data= []

    marque = soup.find('div', {'class':'specs_header'}).get_text(strip=True).split()[0]

    modele = " ".join(soup.find('div', {'class':'specs_header'}).get_text().split()[1:])

    data.extend([marque,modele])
    
    for element in soup.find_all('div', {"class":'rating_percent'}):
        data.append(element.get_text().split()[-1])

    cols = ['Processor', 'Graphics adapter','Memory','Weight','Price']
    c = []

    for element in soup.findAll('div', {'class':'specs'}):
        c.append(element.text)

    w_a = soup.find("span", {'class':'average'}).text
    data.append(w_a)

    h = [c.index(v) if v in c else '-' for v in cols]

    v = []
    for element in soup.findAll('div', {'class':'specs_details'}):
        v.append(" ".join(element.text.split()[:3]))

    for element, value in enumerate(h):
        if value == '-':
            data.append('-')
        elif (element == 2) or (element == 3):
            j = " ".join(v[value].split()[:2])
            data.append(j)
        else:
            data.append(v[value])

    date = soup.find('span', {'class':'intro-date'}).get_text().strip('/')[:10]

    data.append(date)

    return data


In [729]:
# Les colonnes 

def get_cols(soup):
    cols = ['marque','modele']

    for element in soup.find_all('div', {"class":'rating_label'}):
        cols.append(element.get_text())

    cols.extend(["weighted_average",'processor', 'gpu','memory','weight','price', 'review_date'])

    return cols

In [730]:
# Get links 

def get_links(nbre_page):

    links = []
    pages = []
    
    for n in range(0, nbre_page+1):
        token = f'https://www.notebookcheck.net/?&hide_date=1&page={n}&tagArray[]=10&typeArray[]=1&id=48'        
        pages.append(token)

    for page in pages:    
        r = requests.get(page, headers = {'headers':ua}, proxies = {'http':prox}, timeout =5)
        soup = BeautifulSoup(r.text, 'html.parser')

        for element in soup.findAll('a', {'class':'introa_large introa_review'}):
            if element.find('b') != None:
                links.append(element['href'])

    return links

In [732]:
# Procedure

links  = get_links(3)
data = []

for link in links[:50]:
    try:
        soup = get_soup(link)
        d = get_data(soup)
        data.append(d)
    except:
        pass

cols = get_cols(soup)
pd.DataFrame(data, columns=cols)

,marque,modele,Chassis,Keyboard,Pointing Device,Connectivity,Weight,Battery,Display,Games Performance,...,Audio,Camera,Average,weighted_average,processor,gpu,memory,weight,price,review_date
0,realme,C31 (C Series),81%,87%,94%,58%,88%,91%,84%,15%,...,58%,47%,70%,81%,UNISOC T612 8,ARM Mali-G52 MP1,3072 MB,197 g,139 Euro,05/06/2022
1,realme,C35 (C Series),82%,87%,93%,63%,89%,91%,84%,17%,...,67%,56%,72%,82%,UNISOC T616 8,ARM Mali-G57 MP1,4096 MB,189 g,179 Euro,05/04/2022
2,Samsung,Galaxy S22 (Galaxy S Series),92%,92%,93%,94%,91%,89%,91%,91%,...,81%,79%,83%,92%,Samsung Exynos 2200,Samsung Xclipse 920,8192 MB,167 g,849 Euro,05/03/2022
3,realme,GT 2 (GT Series),84%,87%,94%,67%,88%,91%,85%,86%,...,83%,70%,78%,84%,Qualcomm Snapdragon 888,Qualcomm Adreno 660,12288 MB,199.8 g,550 Euro,05/02/2022
4,OnePlus,10 Pro,87%,75%,95%,76%,88%,90%,93%,100%,...,90%,76%,82%,87%,Qualcomm Snapdragon 8,Qualcomm Adreno 730,12288 MB,200.5 g,999 Euro,04/23/2022
5,Google,Pixel 6 Pro (Pixel 6 Series),91%,89%,94%,97%,88%,89%,92%,96%,...,84%,81%,83%,91%,Google Tensor 8,ARM Mali-G78 MP20,12288 MB,210 g,899 Euro,04/13/2022
6,Apple,iPhone SE 2022,83%,88%,94%,69%,93%,89%,90%,93%,...,79%,64%,80%,83%,Apple A15 Bionic,Apple A15 GPU,4096 MB,144 g,519 Euro,04/13/2022
7,Oppo,Find X5 (Find X5 Series),91%,89%,95%,74%,88%,91%,91%,88%,...,80%,78%,82%,91%,Qualcomm Snapdragon 888,Qualcomm Adreno 660,8192 MB,196 g,999 Euro,04/09/2022
8,Nokia,G11 (G Series),82%,87%,93%,63%,89%,92%,83%,15%,...,80%,49%,72%,82%,UNISOC T606 8,ARM Mali-G57 MP1,3072 MB,190 g,160 Euro,04/06/2022
9,Oppo,Find X5 Pro (Find X5 Series),93%,89%,95%,94%,88%,90%,91%,90%,...,86%,80%,83%,93%,Qualcomm Snapdragon 8,"Qualcomm Adreno 730,",12288 MB,221 g,1299 Euro,04/06/2022
